In [1]:
import zipfile

# Unzip the data
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [2]:
!git clone https://github.com/DariusAf/MesoNet.git
!pip install face_recognition

Cloning into 'MesoNet'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 82 (delta 30), reused 26 (delta 25), pack-reused 43 (from 1)
Receiving objects: 100% (82/82), 493.89 KiB | 12.66 MiB/s, done.
Resolving deltas: 100% (33/33), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=c4cbd06cf63b6fe169f5ccb628ee0fe334fa297b81d503582b0a069b64cad7fe
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [ ]:
!cd MesoNet/ && python example.py

2025-01-29 20:54:42.123406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738184082.162752   55570 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738184082.174925   55570 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 20:54:47.335972: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Found 3 images belonging to 3 classes.
Predicted : [[0.04869373]] 
Real class : [2.]
Predicted : [[0.9897476]] 
Real class : [1.]
Predicted : [[0.9977016]] 
Real class : [1.]


In [ ]:
import cv2

backtorgb = cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB)

In [14]:
import numpy as np
import os
import cv2

from MesoNet.classifiers import (
    Meso4
)
from PIL import Image

num_images = 1500 # 1500 for test datasets, 2500 for train datasets

mesonet_classifier = Meso4()
mesonet_classifier.load('MesoNet/weights/Meso4_DF.h5')
# mesonet_classifier.to(device)
mesonet_preds = np.zeros((num_images,))
mesonet_file_num = np.zeros((num_images,))
mesonet_actual = np.zeros((num_images,)) # Change depending on if images are real/df

path = 'data/deepfakeface/real/test/'
files = os.listdir(path)
for i in range(len(files)):
  image = np.array(Image.open(path + files[i]).convert('RGB').resize((256, 256)))
  image = np.expand_dims(image / 255.0, axis=0)
  # Get MesoNet prediction
  mesonet_pred = mesonet_classifier.predict(image)
  # Convert to 0 (real) and 1 (deepfake)
  mesonet_preds[i] = 1 if mesonet_pred[0][0] < 0.5 else 0
  file_num = files[i].split('.')[0][11:] # Change depending on test or train
  mesonet_file_num[i] = int(file_num)

# Save image file numbers and predictions to .npz file
np.savez('mesonet_dff_test_real', mesonet_preds=mesonet_preds, mesonet_actual=mesonet_actual, mesonet_file_num=mesonet_file_num)


In [16]:
from sklearn.metrics import average_precision_score, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

real_data = np.load("mesonet_dff_test_real.npz")
df_data = np.load("mesonet_dff_test_df.npz")

preds = np.concatenate((real_data["mesonet_preds"], df_data["mesonet_preds"]))
actual = np.concatenate((real_data["mesonet_actual"], df_data["mesonet_actual"]))

print(accuracy_score(actual, preds))
print(precision_score(actual, preds))
print(recall_score(actual, preds))
print(f1_score(actual, preds))
print(average_precision_score(actual, preds))


real_data.close()
df_data.close()

# with np.load("real_test_output.npz") as data:
#     print(accuracy_score(np.zeros(1500), np.where(data["mesonet_label_preds"] <= 0.5, 0, 1)))
    # plt.hist(data["mesonet_label_preds"])
    # plt.show()



0.5106666666666667
0.6428571428571429
0.048
0.08933002481389578
0.5068571428571429


In [15]:
from sklearn.metrics import average_precision_score, accuracy_score, precision_score, recall_score, f1_score

data = np.load("mesonet_dff_test_real.npz")
print(accuracy_score(data["mesonet_actual"], data["mesonet_preds"]))
print(np.unique(data["mesonet_file_num"]))
data.close()

0.9733333333333334
[0.000e+00 1.000e+00 2.000e+00 ... 1.497e+03 1.498e+03 1.499e+03]
